<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone

In [0]:
import os

def clone(url, dpath=".", branch="master"):
  """
  Clone remote branch from url into dpath
  branch not provided: clone all branches
  """

  url = url.strip("/")
  rname = os.path.basename(url)
  rpath = os.path.join(dpath, rname)

  # Raise error if dpath inside existing repository
  is_out = !git -C "$dpath" rev-parse
  if not is_out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Clone specific branch
  if branch:
    !git clone --single-branch --branch "$branch" "$url" "$rpath"
  # Clone all branches
  else:
    !git clone "$url" "$rpath"
  os.chdir(rpath)
  
  bname = !git rev-parse --abbrev-ref HEAD
  print("Current")
  print("{branch}\t{directory}".format(branch=bname, directory=os.getcwd()))
  
# clone("https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit")

# Push

In [0]:
from google.colab import files
import json
from urllib.parse import urlsplit

def push(url, branch="master"):
  """
  Push branch to url
  User provides .json file containing text:
  {"username": "<username>", "email": "<email>", "password": "<password>", "name": "<display name>"}
  branch not provided: push all branches
  """
  
  rurl = urlsplit(url)
  
  # Obtain account information
  uploaded = files.upload() # upload .json file
  try:
    [content] = uploaded.values() # bytestring in .json file
    account = json.loads(content)
  finally:
    for p in uploaded: # remove uploaded
      !rm "$p"
  account.setdefault("name", account["username"]) # name defaults to username if not provided
  
  # Construct authorized remote
  rauth = rurl._replace(netloc="{username}:{password}@{hostname}".format(username=account["username"], password=account["password"], hostname=rurl.hostname)) # add username and password

  # Configure git
  name, email = account["name"], account["email"]
  !git config --global user.name "$name"
  !git config --global user.email "$email"
  
  # Push
  remote = rauth.geturl()
  if branch:
    input("Press Enter to push {}...".format(branch))
    !git push "$remote" "$branch"
  else:
    input("Press Enter to push all branches...")
    !git push --all "$remote"

# push("https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit")